In [25]:
# Using Python 3.10
%pip install yfinance
%pip install scikit-learn
%pip install keras
%pip install tensorflow
%pip install py_vollib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for simplejson from https://files.pythonhosted.org/packages/80/7b/45ef1da43f54d209ce2ef59b7356cda13f810186c381f38ae23a4d2b1337/simplejson-3.19.3-cp310-cp310-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
  Created wheel for py_vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62829 sha256=7b9471484508130f5d624c6f6cae1b86a6a8b80b4801b9ca43f1b5fe9a44cb82
  Stored in directory: /Users/john/Library/Caches/pip/wheels/84/0c/fc/b68506eda40cccaeb0013be550ba904d253ec90eae2b156052
  Created wheel for py_lets_be_rational: filename=py_lets_be_rational-1.0.1-

### Simple Model

This model uses a stacked Long Short Term Model (LSTM) architecture with dense layers. The LSTM model structure is suited for time series data. 

The input layer contains the inputs lookback and number of features.

The architecture is as follows:

1. The first LSTM layer contains 50 LSTM units with return sequences enabled, allowing it to output the full sequence.
2. A dropout layer with a rate of 0.2 follows to prevent overfitting.
3. The second LSTM layer also contains 50 LSTM units, but doesn't return sequences, outputting only the final state.
4. Another dropout layer with a rate of 0.2 is applied.
5. A dense layer with 25 units follows, acting as a fully connected layer.
6. The final output layer is a dense layer with a single unit, producing the predicted option price.

The model uses the Adam optimizer with a learning rate of 0.001 and is compiled with mean squared error as the loss function.

This architecture allows the model to capture both long-term dependencies in the time series data through the LSTM layers and make a final prediction through the dense layers.

In [26]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho

In [27]:
# calculating greeks
def calculate_option_price_and_greeks(S, K, T, r, sigma, option_type='c'):
    price = bs(option_type, S, K, T, r, sigma)
    return {
        'price': price,
        'delta': delta(option_type, S, K, T, r, sigma),
        'gamma': gamma(option_type, S, K, T, r, sigma),
        'vega': vega(option_type, S, K, T, r, sigma),
        'theta': theta(option_type, S, K, T, r, sigma),
        'rho': rho(option_type, S, K, T, r, sigma)
    }

In [28]:
# data preparation
def prepare_data(data, lookback=30):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:(i+lookback)])
        y.append(data[i+lookback])
    return np.array(X), np.array(y)

In [36]:
# building the model
def build_model(lookback, n_features):
    input_shape = (lookback, n_features)
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

In [ ]:
def predict_option_prices(tickers, start_date="2010-01-01", end_date="2023-01-01"):
    all_data = []
    for ticker in tickers:
        try:
            stock_data = yf.download(ticker, start=start_date, end=end_date)
            stock_data.index = stock_data.index.tz_localize(None)  # Remove timezone info
            print(f"Downloaded {len(stock_data)} days of stock data for {ticker}")

            ticker_obj = yf.Ticker(ticker)

            # Get all available expiration dates
            expirations = ticker_obj.options

            all_options = []
            for exp in expirations:
                opt = ticker_obj.option_chain(exp).calls
                opt['expirationDate'] = pd.to_datetime(exp)
                all_options.append(opt)

            if not all_options:
                print(f"No options data available for {ticker}")
                continue

            calls = pd.concat(all_options, ignore_index=True)
            calls['Date'] = pd.to_datetime(calls['lastTradeDate']).dt.tz_localize(None)
            print(f"Downloaded {len(calls)} call options for {ticker}")

            # Sort the calls DataFrame by Date
            calls = calls.sort_values('Date')

            # Merge based on date range instead of exact date match
            df = pd.merge_asof(stock_data.reset_index(), calls, left_on='Date', right_on='Date', direction='nearest')
            print(f"After merging, {len(df)} rows remain for {ticker}")

        except Exception as e:
            print(f"Failed to process data for {ticker}: {e}")
            continue

        if df.empty or len(df) < 21:
            print(f"Skipping {ticker} due to insufficient data (less than 21 rows).")
            continue

        # Rest of the processing remains the same
        df['underlying_price'] = df['Close']
        df['strike_price'] = df['strike']
        df['time_to_expiry'] = (df['expirationDate'] - df['Date']).dt.days / 365
        df['risk_free_rate'] = 0.01  # Simplified; consider using actual risk-free rates
        df['implied_volatility'] = df['impliedVolatility']
        df['actual_price'] = df['lastPrice']

        print(f"\nNull values in each column for {ticker}:")
        print(df.isnull().sum())

        df.dropna(inplace=True)
        print(f"After dropping NaN values, {len(df)} rows remain for {ticker}")

        if df.empty or len(df) < 21:
            print(f"Skipping {ticker} after processing due to insufficient data (less than 21 rows).")
            continue

        # Calculate Black-Scholes prices
        bs_results = df.apply(lambda row: calculate_option_price_and_greeks(
            row['underlying_price'], row['strike_price'], row['time_to_expiry'],
            row['risk_free_rate'], row['implied_volatility']), axis=1)

        df['bs_price'] = bs_results.apply(lambda x: x['price'])
        df['delta'] = bs_results.apply(lambda x: x['delta'])
        df['gamma'] = bs_results.apply(lambda x: x['gamma'])
        df['vega'] = bs_results.apply(lambda x: x['vega'])
        df['theta'] = bs_results.apply(lambda x: x['theta'])
        df['rho'] = bs_results.apply(lambda x: x['rho'])

        features = ['underlying_price', 'strike_price', 'time_to_expiry', 'risk_free_rate', 'implied_volatility', 'delta', 'gamma', 'vega', 'theta', 'rho']
        target = 'actual_price'

        scaler = StandardScaler()
        df[features + [target]] = scaler.fit_transform(df[features + [target]])
        all_data.append(df[features + [target]].values)

    if not all_data:
        raise ValueError("No valid data to train the model.")

    combined_data = np.concatenate(all_data, axis=0)
    X, y = prepare_data(combined_data, lookback=30)
    y = y[:, -1]  # The target (actual_price) is the last column

    # Check for NaN or inf values
    if np.isnan(X).any() or np.isinf(X).any() or np.isnan(y).any() or np.isinf(y).any():
        print("Warning: Dataset contains NaN or inf values")
        X = np.nan_to_num(X, nan=0, posinf=1e6, neginf=-1e6)
        y = np.nan_to_num(y, nan=0, posinf=1e6, neginf=-1e6)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Print some statistics about the data
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    print(f"X mean: {np.mean(X)}, X std: {np.std(X)}")
    print(f"y mean: {np.mean(y)}, y std: {np.std(y)}")

    # Correct model building
    lookback = X.shape[1]
    n_features = X.shape[2]
    model = build_model(lookback, n_features)

    # Add early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Fit the model with a smaller learning rate and gradient clipping
    optimizer = Adam(learning_rate=0.0001, clipnorm=1.0)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, 
                        validation_split=0.1, verbose=1, callbacks=[early_stopping])

    print(f"Final training loss: {history.history['loss'][-1]}")
    print(f"Final validation loss: {history.history['val_loss'][-1]}")

    # Plot the learning curves
    # import matplotlib.pyplot as plt
    # plt.figure(figsize=(12, 6))
    # plt.plot(history.history['loss'], label='Training Loss')
    # plt.plot(history.history['val_loss'], label='Validation Loss')
    # plt.title('Model Learning Curves')
    # plt.xlabel('Epoch')
    # plt.ylabel('Loss')
    # plt.legend()
    # plt.show()

    dl_predictions = model.predict(X_test)

    # Inverse transform to get actual prices
    dl_predictions = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], dl_predictions), axis=1))[:, -1]
    actual_prices = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], y_test.reshape(-1, 1)), axis=1))[:, -1]
    bs_prices = scaler.inverse_transform(np.concatenate((X_test[:, -1, :-1], X_test[:, -1, -2].reshape(-1, 1)), axis=1))[:, -1]

    # Calculate errors
    dl_rmse = np.sqrt(np.mean((dl_predictions - actual_prices) ** 2))
    bs_rmse = np.sqrt(np.mean((bs_prices - actual_prices) ** 2))
    dl_mape = np.mean(np.abs((dl_predictions - actual_prices) / actual_prices)) * 100
    bs_mape = np.mean(np.abs((bs_prices - actual_prices) / actual_prices)) * 100

    print(f"Deep Learning RMSE: {dl_rmse}")
    print(f"Black-Scholes RMSE: {bs_rmse}")
    print(f"Deep Learning MAPE: {dl_mape}%")
    print(f"Black-Scholes MAPE: {bs_mape}%")

    return model, dl_predictions, bs_prices, actual_prices

In [61]:
nasdaq_100_tickers = [
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'PYPL', 'NFLX', 'ADBE',
    'GOOG', 'CSCO', 'PEP', 'AVGO', 'COST', 'TXN', 'QCOM', 'INTC', 'HON', 'AMAT',
    'AMD', 'AMGN', 'SBUX', 'MRNA', 'INTU', 'ISRG', 'BKNG', 'ADI', 'MDLZ', 'LRCX',
    'GILD', 'ADP', 'VRTX', 'FISV', 'CHTR', 'ADSK', 'TMUS', 'REGN', 'MU', 'ATVI',
    'ILMN', 'CSX', 'KLAC', 'AEP', 'MNST', 'NXPI', 'PANW', 'CTSH', 'EXC', 'XEL',
    'MELI', 'PAYX', 'IDXX', 'PCAR', 'KDP', 'ROST', 'FAST', 'EA', 'ORLY', 'CDNS',
    'VRSK', 'WBA', 'BIIB', 'DLTR', 'CTAS', 'BIDU', 'MCHP', 'SPLK', 'MAR', 'EBAY',
    'ANSS', 'SWKS', 'ODFL', 'WBD', 'FTNT', 'DDOG', 'TEAM', 'SNPS', 'VRSN', 'ABNB',
    'ALGN', 'ZS', 'DOCU', 'CRWD', 'PDD', 'SGEN', 'OKTA', 'SIRI', 'ZM', 'BMRN',
    'LCID', 'VRSO', 'LULU', 'WDAY', 'CPRT', 'MTCH', 'ASML', 'ZS', 'CHWY', 'GOCO',
]

# nasdaq_100_tickers = [
#     'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'PYPL', 'NFLX', 'ADBE'
# ]

model, dl_predictions, bs_prices, actual_prices = predict_option_prices(nasdaq_100_tickers)


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%ticker%?period1=1262322000&period2=1672549200&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=kb7qiFWSDLL (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x320bb1e70>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))"))
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MSFT']: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%ticker%?period1=1262322000&period2=1672549200&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=kb7qiFWSDLL (Caused by NewConnectionError('<urllib3.connection.HTTPSCo

Failed to process data for AAPL: 'Index' object has no attribute 'tz_localize'
Failed to process data for MSFT: 'Index' object has no attribute 'tz_localize'
Failed to process data for GOOGL: 'Index' object has no attribute 'tz_localize'
Failed to process data for AMZN: 'Index' object has no attribute 'tz_localize'
Failed to process data for META: 'Index' object has no attribute 'tz_localize'
Failed to process data for TSLA: 'Index' object has no attribute 'tz_localize'
Failed to process data for NVDA: 'Index' object has no attribute 'tz_localize'
Failed to process data for PYPL: 'Index' object has no attribute 'tz_localize'
Failed to process data for NFLX: 'Index' object has no attribute 'tz_localize'
Failed to process data for ADBE: 'Index' object has no attribute 'tz_localize'
Failed to process data for GOOG: 'Index' object has no attribute 'tz_localize'
Failed to process data for CSCO: 'Index' object has no attribute 'tz_localize'
Failed to process data for PEP: 'Index' object has 

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['QCOM']: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%ticker%?period1=1262322000&period2=1672549200&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=kb7qiFWSDLL (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x3b2ce8ee0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))"))
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['INTC']: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%ticker%?period1=1262322000&period2=1672549200&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=kb7qiFWSDLL (Caused by NewConnectionError('<urllib3.connection.HTTPSCo

Failed to process data for QCOM: 'Index' object has no attribute 'tz_localize'
Failed to process data for INTC: 'Index' object has no attribute 'tz_localize'
Failed to process data for HON: 'Index' object has no attribute 'tz_localize'
Failed to process data for AMAT: 'Index' object has no attribute 'tz_localize'
Failed to process data for AMD: 'Index' object has no attribute 'tz_localize'
Failed to process data for AMGN: 'Index' object has no attribute 'tz_localize'
Failed to process data for SBUX: 'Index' object has no attribute 'tz_localize'
Failed to process data for MRNA: 'Index' object has no attribute 'tz_localize'
Failed to process data for INTU: 'Index' object has no attribute 'tz_localize'
Failed to process data for ISRG: 'Index' object has no attribute 'tz_localize'
Failed to process data for BKNG: 'Index' object has no attribute 'tz_localize'
Failed to process data for ADI: 'Index' object has no attribute 'tz_localize'
Failed to process data for MDLZ: 'Index' object has no 

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['VRTX']: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%ticker%?period1=1262322000&period2=1672549200&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=kb7qiFWSDLL (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x3b2ce9540>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))"))
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['FISV']: ConnectionError(MaxRetryError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/%ticker%?period1=1262322000&period2=1672549200&interval=1d&includePrePost=False&events=div%2Csplits%2CcapitalGains&crumb=kb7qiFWSDLL (Caused by NewConnectionError('<urllib3.connection.HTTPSCo

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SGEN']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['VRSO']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2010-01-01 -> 2023-01-01)')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Failed to process data for DOCU: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v7/finance/options/DOCU?crumb=kb7qiFWSDLL (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x3b2c2e980>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Downloaded 897 days of stock data for CRWD
Failed to process data for CRWD: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v7/finance/options/CRWD?crumb=kb7qiFWSDLL (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x3b2c2feb0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Downloaded 1117 days of stock data for PDD
Failed to process data for PDD: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v7/finance/options/PDD?crumb=kb7qiFWSDLL (Caused by NewConnectionError

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded 3272 days of stock data for CPRT
Failed to process data for CPRT: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v7/finance/options/CPRT?crumb=kb7qiFWSDLL (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x3b2c49330>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Downloaded 3272 days of stock data for MTCH
Failed to process data for MTCH: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v7/finance/options/MTCH?crumb=kb7qiFWSDLL (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x3b2c482b0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Downloaded 3272 days of stock data for ASML
Failed to process data for ASML: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v7/finance/options/ASML

ValueError: No valid data to train the model.

### Interpretation

1. Root Mean Square Error (RMSE): 0.16471762016671287
   - This measures the standard deviation of the residuals (prediction errors).
   - In the context of option prices, this suggests that on average, the model's predictions deviate from the actual Black-Scholes prices by about $0.16.
   - Given that option prices can vary widely, this error might be considered relatively small, but its significance depends on the typical price range of the options being modeled.

2. Mean Absolute Error (MAE): 0.07295523932601272
   - This represents the average absolute difference between the predicted prices and the Black-Scholes prices.
   - It suggests that, on average, the model's predictions are off by about $0.07 compared to the Black-Scholes prices.
   - This is lower than the RMSE, indicating that there aren't many large outliers in the predictions.

3. Mean Percentage Error: 16.61479047061372%
   - This shows the average percentage difference between the predicted prices and the Black-Scholes prices.
   - A 16.61% error suggests that, on average, the model's predictions deviate from the Black-Scholes prices by about 16.61% in either direction.
   - This error seems relatively high, indicating that while the absolute differences might be small, they represent a significant portion of the option's price.

Interpretation:
- The model appears to be predicting option prices that are reasonably close to the Black-Scholes prices in absolute terms (MAE of $0.07).
- However, the percentage error of 16.61% suggests that these differences are still significant relative to the option prices themselves.
- This could indicate that the model is struggling with certain aspects of option pricing, possibly with very low-priced options where small absolute errors translate to large percentage errors.
- There might be room for improvement, perhaps by adjusting the model architecture, increasing the training data, or incorporating additional features.

To improve the model, you might consider:
1. Increasing the complexity of the LSTM network
2. Adding more relevant features
3. Using a larger dataset or a longer historical period
4. Experimenting with different preprocessing techniques
5. Trying alternative model architectures or ensemble methods

Remember that while the Black-Scholes model is widely used, it also has its limitations. The differences between your model's predictions and Black-Scholes prices could potentially capture some real-world factors that the Black-Scholes model doesn't account for.